Keras Documentation reference-
 https://keras.io/examples/nlp/neural_machine_translation_with_transformer/
 Note:Keras 3.0 is used here
Required Libraries and modules are imported

In [ ]:
import keras

keras.__version__

'2.15.0'

In [ ]:
import re
import string
import random
import pathlib

import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

Loading the data and then generating sentence pairs in tuple form.

In [ ]:
with open('fra.txt') as f:
    lines = f.read().split("\n")[:-1]

text_pairs = []

for line in lines:
    eng, fra = line.split("\t")
    fra = "[start] " + fra + " [end]"

    text_pairs.append((eng, fra))

Viewing some eng-french sentence pairs

In [ ]:
for _ in range(10):
    print(random.choice(text_pairs))

('This is preposterous.', "[start] C'est grotesque. [end]")
('I want to talk to all of them.', '[start] Je veux leur parler à toutes. [end]')
("You aren't as short as I am.", "[start] Vous n'êtes pas aussi petite que moi. [end]")
("He can't be trusted.", '[start] On ne peut pas se fier à lui. [end]')
('You were kind to help me.', "[start] C'était gentil à vous de m'aider. [end]")
('It was a national scandal.', '[start] Ce fut un scandale national. [end]')
('I want to please you.', '[start] Je veux te plaire. [end]')
("That's a myth.", "[start] C'est un mythe. [end]")
('He is my best friend.', "[start] C'est mon meilleur ami. [end]")
("I've been sent to relieve you.", "[start] J'ai été envoyé pour vous soulager. [end]")


Splitting whole text pairs list into training, validation and test pairs

In [ ]:
random.shuffle(text_pairs)

num_val_samples = int(0.15 * len(text_pairs))

num_train_samples = len(text_pairs) - 2 * num_val_samples

train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

167130 total pairs
116992 training pairs
25069 validation pairs
25069 test pairs


Vectorizing the text data
Usinf two instances of the TextVectorization layer to vectorize the text data (one for English and one for French), that is to say, to turn the original strings into integer sequences where each integer represents the index of a word in a vocabulary.

The English layer will use the default string standardization (strip punctuation characters) and splitting scheme (split on whitespace), while the French layer will use a custom standardization,



In [ ]:
strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)

    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length,
)

fra_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization,
)


Creating both eng and french vocabularies using training data

In [ ]:
train_eng_texts = [pair[0] for pair in train_pairs]
train_fra_texts = [pair[1] for pair in train_pairs]

eng_vectorization.adapt(train_eng_texts)
fra_vectorization.adapt(train_fra_texts)

Checking the english vocabulary

In [ ]:
vocab_inp_size = len(eng_vectorization.get_vocabulary())

vocab_inp_size

12842

In [ ]:
eng_vectorization.get_vocabulary()[:15]

['',
 '[UNK]',
 'i',
 'you',
 'to',
 'the',
 'a',
 'is',
 'tom',
 'that',
 'he',
 'do',
 'of',
 'it',
 'this']

Checking the french vocabulary

In [ ]:
vocab_out_size = len(fra_vectorization.get_vocabulary())

vocab_out_size

15000

In [ ]:
fra_vectorization.get_vocabulary()[:15]

['',
 '[UNK]',
 '[start]',
 '[end]',
 'je',
 'de',
 'pas',
 'que',
 'ne',
 'à',
 'la',
 'le',
 'vous',
 'il',
 'tom']

Next, formatting our datasets.

At each training step, the model will seek to predict target words N+1 (and beyond) using the source sentence and the target words 0 to N.

As such, the training dataset will yield a tuple (inputs, targets), where:

- inputs is a dictionary with the keys encoder_inputs and decoder_inputs.
- encoder_inputs is the vectorized source sentence and decoder_inputs is the target sentence "so far", that is to say, the words 0 to N used to predict word N+1 (and beyond) in the target sentence.
- target is the target sentence offset by one step: it provides the next words in the target sentence — what the model will try to predict.

In [ ]:
def format_dataset(eng, fra):
    eng = eng_vectorization(eng)
    fra = fra_vectorization(fra)

    # ({english tokens, french tokens}, french tokens shifted by 1)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": fra[:, :-1],
        },
        fra[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, fra_texts = zip(*pairs)

    eng_texts = list(eng_texts)
    fra_texts = list(fra_texts)

    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, fra_texts))

    dataset = dataset.batch(batch_size)

    dataset = dataset.map(format_dataset)

    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [ ]:
 inputs, targets = next(iter(train_ds))

 print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
 print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
 print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


Checking the inputs and targets..Note that target is offset by one step from decoder_inputs

In [ ]:
inputs["encoder_inputs"]

<tf.Tensor: shape=(64, 20), dtype=int64, numpy=
array([[  10, 3326,   16, ...,    0,    0,    0],
       [  41,   30,  716, ...,    0,    0,    0],
       [   2,  282,    2, ...,    0,    0,    0],
       ...,
       [   7,    9,    6, ...,    0,    0,    0],
       [  47,    6,  767, ...,    0,    0,    0],
       [   3,  129,    6, ...,    0,    0,    0]])>

In [ ]:
inputs["decoder_inputs"]

<tf.Tensor: shape=(64, 20), dtype=int64, numpy=
array([[  2,  13,  37, ...,   0,   0,   0],
       [  2,  12, 259, ...,   0,   0,   0],
       [  2, 133,   8, ...,   0,   0,   0],
       ...,
       [  2,  89,  16, ...,   0,   0,   0],
       [  2,  29,  16, ...,   0,   0,   0],
       [  2,  19, 117, ...,   0,   0,   0]])>

In [ ]:
targets

<tf.Tensor: shape=(64, 20), dtype=int64, numpy=
array([[  13,   37,    1, ...,    0,    0,    0],
       [  12,  259,    6, ...,    0,    0,    0],
       [ 133,    8,    6, ...,    0,    0,    0],
       ...,
       [  89,   16, 1967, ...,    0,    0,    0],
       [  29,   16, 9255, ...,    0,    0,    0],
       [  19,  117,   38, ...,    0,    0,    0]])>

Our sequence-to-sequence Transformer consists of a TransformerEncoder and a TransformerDecoder chained together. To make the model aware of word order, we also use a PositionalEmbedding layer.

The source sequence will be pass to the TransformerEncoder, which will produce a new representation of it. This new representation will then be passed to the TransformerDecoder, together with the target sequence so far (target words 0 to N). The TransformerDecoder will then seek to predict the next words in the target sequence (N+1 and beyond).

A key detail that makes this possible is causal masking (see method get_causal_attention_mask() on the TransformerDecoder). The TransformerDecoder sees the entire sequences at once, and thus we must make sure that it only uses information from target tokens 0 to N when predicting token N+1 (otherwise, it could use information from the future, which would result in a model that cannot be used at inference time).



In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)

        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads

        self.attention = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )

        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation = "relu"),
                layers.Dense(embed_dim),
            ]
        )

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

        self.supports_masking = True

    def call(self, inputs, mask = None):
        attention_output = self.attention(query = inputs, value = inputs, key = inputs)

        proj_input = self.layernorm_1(inputs + attention_output)

        proj_output = self.dense_proj(proj_input)

        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()

        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )

        return config

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)

        self.token_embeddings = layers.Embedding(
            input_dim = vocab_size, output_dim = embed_dim
        )

        self.position_embeddings = layers.Embedding(
            input_dim = sequence_length, output_dim = embed_dim
        )

        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start = 0, limit = length, delta = 1)

        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)

        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask = None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()

        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )

        return config

Decoder is defined

In [ ]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)

        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads

        self.attention_1 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )

        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation = "relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()

        self.add = layers.Add()  # instead of `+` to preserve mask

        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask = None):
        attention_output_1 = self.attention_1(
            query = inputs, value = inputs, key = inputs, use_causal_mask = True
        )
        out_1 = self.layernorm_1(self.add([inputs, attention_output_1]))

        attention_output_2 = self.attention_2(
            query = out_1,
            value = encoder_outputs,
            key = encoder_outputs,
        )
        out_2 = self.layernorm_2(self.add([out_1, attention_output_2]))

        proj_output = self.dense_proj(out_2)

        return self.layernorm_3(self.add([out_2, proj_output]))

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config

Assembling the end to end model

In [ ]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

# Encoder
encoder_inputs = keras.Input(shape = (None,), dtype = "int64", name = "encoder_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)

encoder = keras.Model(encoder_inputs, encoder_outputs)

# Decoder
decoder_inputs = keras.Input(shape = (None,), dtype = "int64", name = "decoder_inputs")

encoded_seq_inputs = keras.Input(shape = (None, embed_dim), name = "decoder_state_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)

x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation = "softmax")(x)

decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])

# Transformer
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name = "transformer"
)


We'll use accuracy as a quick way to monitor training progress on the validation data. Note that machine translation typically uses BLEU scores as well as other metrics, rather than accuracy.

In [ ]:
epochs = 30  # This should be at least 30 for convergence

transformer.summary()

transformer.compile(
    "rmsprop", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]
)

transformer.fit(train_ds, epochs = epochs, validation_data = val_ds)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['encoder_inputs[0][0]']      
 sitionalEmbedding)                                                                               
                                                                                                  
 decoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                      

In [ ]:
fra_vocab = fra_vectorization.get_vocabulary()
fra_index_lookup = dict(zip(range(len(fra_vocab)), fra_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"


    for i in range(max_decoded_sentence_length):
        # Tokenize the sentence generated so far
        tokenized_target_sentence = fra_vectorization([decoded_sentence])[:, :-1]

        # Predict the next word in the sequence using input sentence and
        # tokenized representation of sentence so far
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        # Look up the index of the current predicted word
        sampled_token_index = np.argmax(predictions[0, i, :])

        # Find the corresponding token from the vocabulary
        sampled_token = fra_index_lookup[sampled_token_index]

        # Add the predicted word to the decoded text so far
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

In [ ]:
test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(30):
    ip = random.choice(test_eng_texts)
    print("input :",ip)
    print("translation :", decode_sequence(ip))

input : We have broken off relations with them.
translation : [start] nous avons cassé les [UNK] des mots en panne [end]
input : I spilled the milk.
translation : [start] jai frappé le lait [end]
input : Why exactly did you do that?
translation : [start] pourquoi avezvous fait cela exactement [end]
input : Almost all of the dogs are alive.
translation : [start] presque tous les chiens sont vivant [end]
input : He donated countless pieces to the museum.
translation : [start] il a fait semblant en [UNK] le musée [end]
input : This book is much more interesting than that one.
translation : [start] ce livre est bien plus intéressant que celleci [end]
input : It would never have occurred to me to say anything like that.
translation : [start] Ça ne me ferait jamais là cela lui à dire quelque chose [end]
input : They forgot to lock the door.
translation : [start] elles ont oublié de fermer la porte à clé [end]
input : He was angry that I had insulted him.
translation : [start] il était en col

In [ ]:
for test_pair in test_pairs[:30]:
    input_sentence = test_pair[0]
    reference_sentence = test_pair[1]

    translated_sentence = decode_sequence(input_sentence)
    translated_sentence = (
        translated_sentence.replace("[PAD]", "")
        .replace("[start]", "")
        .replace("[end]", "")
        .strip()
    )

    reference_sentence = (
        reference_sentence.replace("[PAD]", "")
        .replace("[start]", "")
        .replace("[end]", "")
        .strip()
    )

    print("English: ", input_sentence)
    print("Translation: ", reference_sentence)
    print("Prediction: ", translated_sentence)

English:  We went to see our neighbors.
Translation:  Nous sommes allés voir nos voisins.
Prediction:  nous sommes allés voir nos voisins
English:  I'm not busy anymore.
Translation:  Je ne suis plus occupé.
Prediction:  je ne suis plus occupé
English:  We should save money for a rainy day.
Translation:  Nous devrions mettre le l'argent de côté.
Prediction:  nous devrions sauver largent pour un jour de pluie
English:  Tom is looking for that.
Translation:  Tom le cherche.
Prediction:  tom cherche cela
English:  I wonder who took it.
Translation:  Je me demande qui l'a prise.
Prediction:  je me demande qui la pris
English:  This safe is for keeping valuables.
Translation:  Ce coffre est destiné à conserver les objets de valeur.
Prediction:  cest en sécurité pour garder les valeur de valeur
English:  Is there much demand for these goods?
Translation:  Y a-t-il beaucoup de demande pour ces marchandises ?
Prediction:  y atil vraiment [UNK] des marchandises pour ces [UNK]
English:  I fell a